# <center> Notebook para aprender a ajustar datos con curve_fit</center>

In [4]:
from scipy.stats import norm
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import pandas as pd
import numpy as np

Primero voy a generar datos random con distribucion normal. Esto lo hace la funcion norm.rvs (normal distribution(norm) Random Variable Sample (rvs)). Los datos van a tener una distribucion con un valor promedio (mu) de 0.7 y una desviacion estandar (sigma) de 0.1. Genero 100 datos random con esta distribucion.

In [2]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm
mu = 0.7  # Mean
sigma = 0.1  # Standard deviation (scale parameter)
size = 1000  # Number of random numbers to generate

# Generate random numbers from a normal distribution
data = norm.rvs(loc=mu, scale=sigma, size=size) # Esto es un array de numpy, "una lista",  con los datos random.

# Grafico el histograma para que veas que mas o menos responde a lo que le pedimos
fig = go.Figure()
fig.add_traces(go.Histogram(x = data))

In [60]:
dens_prob, bin_edges = np.histogram(data, # Aca van los datos crudos como la lista de tamaño de paso
                                 bins = 20, # Cuantos bines vas a usar. Hay una ecuacion para determinar cuantos bines usar.
                                 density = True, # El parametro density devuelve la densidad de probabilidad. PAra ajustar con una PDF este parametro tiene que ser True
                                 )
print(dens_prob) 
print(bin_edges)

[0.02949249 0.         0.08847747 0.08847747 0.47187982 1.00274462
 1.41563946 2.12345919 2.65432399 4.18793341 4.12894843 3.71605359
 2.97874137 2.18244417 1.79904182 1.327162   0.67832724 0.32441738
 0.20644742 0.08847747]
[0.33502066 0.3689276  0.40283454 0.43674148 0.47064842 0.50455536
 0.53846229 0.57236923 0.60627617 0.64018311 0.67409005 0.70799699
 0.74190392 0.77581086 0.8097178  0.84362474 0.87753168 0.91143862
 0.94534555 0.97925249 1.01315943]


Nota: La desnidad de probabilidad es lo que multiplicado por el ancho del bin ( barra de histograma) te da la probabilidad de que un numero random sea un numero entre el intervalo del bin
Para poder ajustar una densidad de probabilidad (probability density function o PDF) hay que darle puntos (x,y) a curve_fit. Como numpy arroja un valor de más en el eje x, porque da los valores de los bordes de los bines, hay que rehacerlos. 

In [55]:
delta_bin = (bin_edges[1]-bin_edges[0])/2 # Calculo la distancia que hay entre dos bordes de bin y lo divido a la mitad. Esto es lo que tengo que correr los valores de bin_edges
new_bins = bin_edges[:-1] + delta_bin # me quedo con los valores de bin_edges menos el ultimo. Luego los corro medio bin. De esta manera obtengo el punto medio de cada bin.
print(new_bins)

[0.35197413 0.38588107 0.41978801 0.45369495 0.48760189 0.52150882
 0.55541576 0.5893227  0.62322964 0.65713658 0.69104352 0.72495045
 0.75885739 0.79276433 0.82667127 0.86057821 0.89448515 0.92839209
 0.96229902 0.99620596]


Voy a hacer un gráfico de barras con new_bins y dens_prob para ver si queda como un histograma.

In [61]:
fig = go.Figure()
fig.add_traces(go.Bar(x = new_bins,
                      y = dens_prob))
#fig.add_traces(go.Histogram(x = data)) #si queres comparar tu histograma con el que da plotlyaca podes usarcomentar esta linea

## Ajuste
Ahora vamos a ajustar el histograma con el método curve_fit de scipy.optimize. Esta funcion necesita una funcion (gauss) cuyo primer argumento sea "x" y luego los parametros que se te canten. En algun momento de tu vida va s a tener que ajustar otras cosas. Esta es la funcion que deberias cambiar.

In [23]:
def gauss(x,m,s):
    #La funcion PDF que esta en la página https://en.wikipedia.org/wiki/Normal_distribution
    return (1/(s*(2*np.pi)**0.5))*np.exp(-0.5*((x-m)/s)**2)

In [69]:
popt, pcov = curve_fit(gauss, xdata = new_bins, ydata = dens_prob)
print('Parámetros optimos:',popt) # estos son los Parametros OPTimos. Es un array con los parametros optimos en el orden en el que estan en la funcion. en este caso primero "m" y luego "s"
print('parametros iniciales:', mu,sigma)

Parámetros optimos: [0.695886   0.10080551]
parametros iniciales: 0.7 0.1


In [67]:
#Para obtener los errores de estos parametros:
errs = np.sqrt(np.diag(pcov))
print('Errores de los parametros:',errs)

Errores de los parametros: [0.00331818 0.00270989]


## Grafico de ajuste con datos

In [68]:
fig = go.Figure()
fig.add_traces(go.Bar(x = new_bins,
                      y = dens_prob,
                      name = 'Histograma'))
fig.add_traces(go.Scatter(x = new_bins,
                          y = gauss(new_bins,popt[0],popt[1]), # Explico abajo que hago aca
                          name = 'Ajuste')) 

La funcion que definimos gauss es capaz de devolver un valor si le asignamos valores a los parametros y a x. Si x resulta ser una lista de valores lo que nos va a devolver es una lista de valores que serian los correspondientes a los de la campana de gauss. popt[0] es el valor de mu que arroja el ajuste. popt[1] es el valor de sigma que arroja el ajuste.

# Estetica de los graficos
Te dejo una funcion para darle la estetica de papers a los graficos para que juegues un poco. (no recuerdo si te la pase o no)

In [84]:
def estetica(figura,x_name = '',y_name = '',title = '',w = 400,h = 325):
    '''
    Esta funcion sirve para darle una sierta estetica a las graficas. Solo modifica parametros de ejes y layout de las figuras de plotly. 
    IMPORTANTE: Luego de utilizar esta funcion es importante usar fig.show()
    figura: plotly object ->Es el objeto go.Figure(). la mayoria de las veces es "fig" (sin las comillas).
    x_name: str -> Es el nombre del eje X.
    y_name: str -> Es elnombre del eje Y.
    title: str -> Titulo de la figura.
    w: int -> ancho de la figura en pixeles.
    h: int -> alto de la figura en pixeles. 

    '''
    
    figura.update_xaxes(showgrid=True, # Muestra una grilla de fondo
                        showline=True, # Muestra una linea en el borde izquierdo de la figura
                        linewidth=2, # Ancho de la linea de arriba
                        linecolor='black', #color de la linea
                        mirror=True, # Tambien la pone a la derecha. Todo esto es para recuadrar la imagen. Queda lindo.
                        title = x_name,
                        )
    # Los parametros de update_yaxes son similares a los de x_axes
    figura.update_yaxes(showgrid=True,showline=True, linewidth=2, linecolor='black', mirror=True,title =y_name)


    figura.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),# Para que no quede tanto margen en la figura
        plot_bgcolor="white", 
        template="plotly_white", 
        title=title,
        autosize=False,
        width=w,
        height=h,
        # Si queres definir el rango de los ejes descomenta las lineas de abajo
        #yaxis = dict(range=[0,1]), 
        #xaxis = dict(range=[0,1]),
        font= dict(size = 22), # Fuente de los ejes
        title_font = dict(size =20), # La fuente del titulo. Tiene mas parametros, solo puse la fuente
        # Acá trabajo con la leyenda: 
        legend=dict(orientation = 'v', # Puede ir "h" si queres que la legenda quede horizontal. la "v" es de vertical
                    font = dict(size = 22), #fuente de la leyenda. solo Puse el tamañod e la misma, pero hay mas cosas para modificar si se quiere
                     # Las dos lineas de abajo dicen que la parte de arriba (top) tiene que estar 95% ariiba de la imagen. Siendo 1 arriba del todo y 0 abajo del todo
                     yanchor="top", # opciones: top, bottom ó center
                     y=0.95,
                     # Las dos lineas de abajo dicen que la parte derecha de la leyenda tienen que estar a la derecha de la figura. 1 es todo a la derecha y 0 todo a la izquierda
                     xanchor="right", # Opciones: right, left o center
                     x=1,
                     bgcolor='rgba(255,255,255,0)', # Este es el color del fondo. Los numeros varian entre 0 y 255. El primer color es el rojo, luego el verde, despues el azul
                      # y por ultimo es la transparencia. Este ultimo varia entre 0 y 1 siendo 0 transparente y 1 opaco
                   )
    )

In [83]:
fig = go.Figure()
fig.add_traces(go.Bar(x = new_bins,
                      y = dens_prob,
                      name = 'Histograma'))
fig.add_traces(go.Scatter(x = new_bins,
                          y = gauss(new_bins,popt[0],popt[1]), # Explico abajo que hago aca
                          name = 'Ajuste')) 
estetica(fig,x_name = 'nombre X',y_name = 'nombre Y', title = 'caca con cebolla',w = 800,h = 600)
fig.show()

## Cantidad de bines en histograma
Esta es una funcion que usa la formula de Freedman-Diaconis para determinar la cantidad de bines necesarios para describir con un histograma una serie de valores.
<a>https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule</a>

In [85]:
def num_bines(data):
    # La cantidad optima de bines en un histograma viene dada por la regla de Freedman-Diaconis. 
    data =pd.Series(data)
    q_75 = data.quantile(0.75)
    q_25 = data.quantile(0.25)
    max_data = data.max()
    min_data = data.min()
    num_bins = (max_data-min_data)/(2*(q_75-q_25)*len(data)**(-1/3))
    return(int(num_bins))

In [87]:
print('Cantidad de bines necesarios para describir los valores del ejemplo:', num_bines(data))

Cantidad de bines necesarios para describir los valores del ejemplo: 25
